# Implementing tile overlap

The example that this process is based off is only 2D image. 

I have a 4D stack so I need to separate each layer and tile individually.

Now tiling over time and z stack, but testing to see if order of iteration matters. 

In [27]:
import napari
import matplotlib.pyplot as plt
import glob
import os
import octopusheavy as octo
import pandas as pd
from tqdm.auto import tqdm
from functools import partial
from octopusheavy import tile
from skimage.io import imread
import dask.array as da

#### Find images

In [28]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
# image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/r03c03f01p01/'
fns = glob.glob(os.path.join(image_dir, '*.tiff'))
# fns = glob.glob(os.path.join(image_dir, f'r0{row}c0{col}*.tiff'))
print(len(fns), 'image files found')

138402 image files found


### Loading metadata

In [29]:
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
df = octo.utils.read_harmony_metadata(metadata_fn)

Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

### Assay layout metadata

In [35]:
assay_metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = octo.utils.read_harmony_metadata(assay_metadata_fn)

IndexError: list index out of range

In [37]:
import xml.etree.ElementTree as ET

In [38]:
xml_data = open(assay_metadata_fn, 'r').read()
root = ET.XML(xml_data)

In [39]:
root

<Element '{http://www.perkinelmer.com/PEHH/HarmonyV5}AssayLayout' at 0x7f904efe9ea0>

In [51]:
for i in [child for child in root if "Layer" in child.tag]:
    for j in i:
        if j != ' ':
            print(j.text)


Strain
string

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      
Compound
string

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

      

 

In [ ]:
### read xml metadata file
xml_data = open(assay_metadata_fn, 'r').read()
root = ET.XML(xml_data)
### extract the metadata from the xml file
images_metadata = [child for child in root if "Images" in child.tag][0]
### create an empty list for storing individual image metadata
all_images_dicts = list()
### iterate over every image entry extracting the metadata
for image_metadata in tqdm(images_metadata, total = len(images_metadata),
                            desc = 'Extracting HarmonyV5 metadata'):
    ### create empty dict to store single image metadata
    single_image_dict = dict()
    ### iterate over every metadata item in that image metadata
    for item in image_metadata:
        ### get column names from metadata
        col = item.tag.replace('{http://www.perkinelmer.com/PEHH/HarmonyV5}','')
        ### get metadata
        entry = item.text
        ### make dictionary out of metadata
        single_image_dict[col] = entry
    ### append that image metadata to list of all images
    all_images_dicts.append(single_image_dict)
### create a dataframe out of all metadata
df = pd.DataFrame(all_images_dicts)

In [30]:
row_col_list = list()
for index, row in tqdm(df.iterrows(), total = len(df)):
    row_col_list.append(tuple((int(row['Row']), int(row['Col']))))
row_col_list = list(set(row_col_list))
for n, i in enumerate(row_col_list):
    print('Position index and (row,column):', n, i)

  0%|          | 0/113400 [00:00<?, ?it/s]

Position index and (row,column): 0 (3, 4)
Position index and (row,column): 1 (4, 3)
Position index and (row,column): 2 (4, 9)
Position index and (row,column): 3 (3, 7)
Position index and (row,column): 4 (5, 4)
Position index and (row,column): 5 (4, 6)
Position index and (row,column): 6 (3, 10)
Position index and (row,column): 7 (5, 7)
Position index and (row,column): 8 (6, 5)
Position index and (row,column): 9 (6, 8)
Position index and (row,column): 10 (4, 5)
Position index and (row,column): 11 (3, 3)
Position index and (row,column): 12 (3, 9)
Position index and (row,column): 13 (5, 6)
Position index and (row,column): 14 (4, 8)
Position index and (row,column): 15 (3, 6)
Position index and (row,column): 16 (5, 9)
Position index and (row,column): 17 (6, 4)
Position index and (row,column): 18 (6, 7)
Position index and (row,column): 19 (4, 7)
Position index and (row,column): 20 (3, 5)
Position index and (row,column): 21 (4, 4)
Position index and (row,column): 22 (4, 10)
Position index and 

## Get dimensionality of image volume

In [5]:
channel_IDs = df['ChannelID'].unique()
plane_IDs = df['PlaneID'].unique()
timepoint_IDs = df['TimepointID'].unique()

## Set mosaic parameters

The `chunk_fraction` is how many sections you want one slice cut up into (has to be a square) and the `_load_image` partial function can include any image transformations you wish (border crop or background removal etc).

In [6]:
chunk_fraction = 49
load_transform_image = partial(tile.load_image, transforms=[])#input_transforms)

## Set segmentation parameters

In [7]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

model = models.Cellpose(gpu=True, model_type='cyto')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Wed Dec 14 14:50:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:65:00.0  On |                  Off |
| 30%   35C    P8    32W / 300W |  22990MiB / 49140MiB |     34%      Default |
|                               |                      |                  N/A |
+

In [ ]:
def segment(img):
    masks, flows, styles, diams = model.eval(img, diameter=200, channels=[0,0],
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

# Compile and segment at the same time...? 

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
lazy_hack = {1:(slice(4032, 6048), slice(2016, 4032)),
            2:(slice(0, 2016), slice(0, 2016)),
            3:(slice(0, 2016), slice(2016, 4032)),
            4:(slice(0, 2016), slice(4032, 6048)),
            5:(slice(2016, 4032), slice(4032, 6048)),
            6:(slice(2016, 4032), slice(2016, 4032)),
            7:(slice(2016, 4032), slice(0, 2016)),
            8:(slice(4032, 6048), slice(0, 2016)),
            9:(slice(4032, 6048), slice(4032, 6048)),
            }
import re
regex = re.compile('f([0-9]*)')
from skimage.io import imsave

In [20]:
for row, col in tqdm(reversed(row_col_list), desc = 'Position progress'):
    print(row, col)

Position progress:   0%|          | 0/28 [00:00<?, ?it/s]

3 4
4 3
4 9
3 7
5 4
4 6
3 10
5 7
6 5
6 8
4 5
3 3
3 9
5 6
4 8
3 6
5 9
6 4
6 7
4 7
3 5
4 4
4 10
3 8
5 5
5 8
6 6
6 9


In [21]:
for row, col in tqdm(reversed(row_col_list), desc = 'Position progress'):
    print(row, col)

Position progress: 0it [00:00, ?it/s]

6 9
6 6
5 8
5 5
3 8
4 10
4 4
3 5
4 7
6 7
6 4
5 9
3 6
4 8
5 6
3 9
3 3
4 5
6 8
6 5
5 7
3 10
4 6
5 4
3 7
4 9
4 3
3 4


# running in reverse

In [54]:
col

'8'

In [55]:
z_mask_stack = []
t_mask_stack = []
# images = dict()
# masks = dict()
# for row, col in tqdm(row_col_list, desc = 'Position progress'):
# for row, col in tqdm(reversed(row_col_list), desc = 'Position progress'):
    ### define row and col
#     row, col = str(row), str(col)
row, col = '3', '8'
### clear empty arrays for organsing into dask arrays
t_stack = []
t_mask_stack = []
### iterate over each individual image slice, tiling together
for time in tqdm(timepoint_IDs, leave = False, desc = 'Timepoint progress'):
    c_stack = []
    for channel in tqdm(channel_IDs, leave = False, desc = 'Channel progress'):
        z_stack = []
        if channel == '1':
            z_mask_stack = []
        for plane in tqdm(plane_IDs, leave = False, desc = 'Z-slice progress'):
            frame, chunk_info = tile.stitch(load_transform_image,
                                df, 
                                image_dir, 
                                time, 
                                plane, 
                                channel, 
                                row, 
                                col, 
                                chunk_fraction)
            ### if we're iterating over the GFP channel then segment that
            if channel == '1':



                masks_frame = segment(frame)
                ### convert to chunked dask array
                masks_frame = da.from_array(masks_frame, chunks=frame.chunksize)
                ### append to z stack
                z_mask_stack.append(masks_frame)
                ### use chunk information to iterative save out individual tiles
                for chunk in chunk_info:
                    fn = chunk.fuse_info['file']
#                         x, y = chunk.fuse_info['transform'][0:2,2].astype(int)
#                         crops = tuple((slice(x, x+2016), slice(y, y+2016)))
                    ID = int((regex.findall(fn))[0])
                    crops = lazy_hack[ID]
                    save_out_mask = masks_frame[crops]
                    new_fn = fn.replace('ch1', 'ch99')
                    if new_fn == fn:
                        raise Exception(f"Tried overwriting file {fn}") 
                    else:
                        imsave(new_fn, save_out_mask, check_contrast=False)
            ### collect stitched frames together into time stack
            z_stack.append(frame)
        ### stack channel series together (images)
        c_stack.append(z_stack)
    ### stack together timewise
    t_stack.append(c_stack)
    t_mask_stack.append(z_mask_stack)
### stack stitched dask arrays together into multidim image volumes
images[(int(row), int(col))] = da.stack([da.stack(c_stack, axis = 0) for c_stack in t_stack])
masks[(int(row), int(col))] = da.stack(t_mask_stack)

Timepoint progress:   0%|          | 0/75 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/2 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/2 [00:00<?, ?it/s]

...

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/2 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [52]:
new_fn

'/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/r03c08f09p01-ch99sk35fk1fl1.tiff'

In [10]:
frame, chunk_info = tile.stitch(load_transform_image,
                                    df, 
                                    image_dir, 
                                    time, 
                                    plane, 
                                    channel, 
                                    row, 
                                    col, 
                                    chunk_fraction)

/home/dayn/analysis/octopusheavy-reader/octopusheavy/tile.py:252: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords['PositionXPix'] = (coords['PositionX'].astype(float))/(coords['ImageResolutionX']).astype(float)
/home/dayn/analysis/octopusheavy-reader/octopusheavy/tile.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords['PositionYPix'] = (coords['PositionY'].astype(float))/(coords['ImageResolutionY']).astype(float)
/home/dayn/analysis/octopusheavy-reader/octopusheavy/tile.py:282: ShapelyDepreca

In [12]:
from skimage.io import imshow

In [13]:
imshow(frame)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/skimage/io/_plugins/matplotlib_plugin.py:150: UserWarning: Low image data range; displaying image with stretched contrast.
  lo, hi, cmap = _get_display_range(image)


In [15]:
row, col, time, channel, plane

('3', '4', '0', '1', '1')

In [19]:
filtered_df = df[(df['TimepointID'] == time)
                   &(df['PlaneID'] == plane)
                   &(df['ChannelID'] == channel)
                   &(df['Row'] == row)
                   &(df['Col'] == col)
                    ]

# Check segment

In [ ]:
v = napari.Viewer()
v.add_image(images[(3, 4)],
            channel_axis=1,
            name=["macrophage", "mtb"],
            colormap=["green", "magenta"],
            )  
# v.add_image(images[(3, 4)])
v.add_labels(masks[(3, 4)])